In [5]:
import os
# from labelbox import Client
import requests
import pandas as pd

# import utils

from datetime import datetime


In [6]:
## NUS: Modify this to load directly from JSON
df_label_data = pd.read_json("../datasets/persent_annotated/per_sent_labelbox.json",lines=True, orient='records')

In [7]:
df_skipped = df_label_data[df_label_data[ 'Skipped']]
print(f"There are {len(df_skipped)} sample are skipped")

#remove skipped
df_labelled = df_label_data[~df_label_data[ 'Skipped']].reset_index(drop=True)
print(f"There are {len(df_labelled)} labels for {df_labelled['DataRow ID'].nunique()} samples")


There are 14 sample are skipped
There are 13522 labels for 4512 samples


In [8]:
df = pd.json_normalize(df_labelled['Label'])

# Article Level Labels

In [9]:
temp1 = df[['classifications']].explode('classifications').dropna().reset_index()
temp1 = temp1.join(pd.json_normalize(temp1['classifications']))
temp1.rename({'answer.value': "answer_value"}, axis=1, inplace=True)
temp1 = temp1[temp1['value'] != 'overall_article_target_aspect_category']


def convert_answer(x):
    if x.value == 'article_target_tags_sentiment':
        return x.answer[0]['value']
    elif x.value == "other_please_specify":
        return x.answer[0]['value']
    elif x.value == "other_please_specify_social_issue":
        return x.answer
    else:
        return x.answer_value


temp1["answer.value"] = temp1.apply(convert_answer, axis=1)
temp1 = temp1[['index', 'value', 'answer.value']]
temp1.drop_duplicates(subset=['index', 'value'], inplace=True)
temp1 = temp1.pivot(index='index', columns='value', values='answer.value')
temp1.columns.rename(None, inplace=True)
temp1.rename(columns={'article_target_tags_sentiment': 'TRUE_SENTIMENT',
                      'other_please_specify': "sentiment_other",
                      "other_please_specify_social_issue": "ASPECT-other"}, inplace=True)


In [10]:
df_article = df_labelled[['ID', 'External ID']].join(temp1)
df_article.set_index(['ID'], inplace=True)
sent_cols_dict = {item: "ASPECT_SENTIMENT-"+item[10:]
                  for item in df_article.columns if item.startswith('sentiment')}
df_article.rename(columns=sent_cols_dict, inplace=True)
sent_cols = list(sent_cols_dict.values())
df_article_sentiment = df_article[['TRUE_SENTIMENT', *sent_cols]].replace({"positive": 2,
                                                                           'neutral': 1,
                                                                           'negative': 0,
                                                                           'mixed_positive_negative': 3,
                                                                           'mixed': 3}).fillna(-1)
df_article = df_article[['External ID', "ASPECT-other"]].join(df_article_sentiment)

aspect_sent_main_cols = [item for item in df_article.columns
                         if item.startswith('ASPECT_SENTIMENT') and "other" not in item]

aspect_sent_other_cols = ['ASPECT_SENTIMENT-other',
                          'ASPECT-other', 'ASPECT_SENTIMENT-other_not_a_social_issue']

df_article = df_article[['External ID', 'TRUE_SENTIMENT', *aspect_sent_main_cols, *aspect_sent_other_cols]].reset_index()
df_article 


,ID,External ID,TRUE_SENTIMENT,ASPECT_SENTIMENT-corruption,ASPECT_SENTIMENT-crime_justice_system,ASPECT_SENTIMENT-digital_online,ASPECT_SENTIMENT-economic_issues,ASPECT_SENTIMENT-education,ASPECT_SENTIMENT-environmental,ASPECT_SENTIMENT-public_health,ASPECT_SENTIMENT-religion,ASPECT_SENTIMENT-social_inequality_human_rights,ASPECT_SENTIMENT-terrorism,ASPECT_SENTIMENT-work_occupation,ASPECT_SENTIMENT-other,ASPECT-other,ASPECT_SENTIMENT-other_not_a_social_issue
0,cl7pdl4ot20m207v00xnz99ob,1692,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,NaN,-1.0
1,cl7pdlzesgulx07xmfdc571a6,1608,3.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,-1.0
2,cl7pjzver3ilt08w80h0fgew2,4023,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,NaN,-1.0
3,cl809q5010dsj073d2pcmgtyc,4272,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,NaN,-1.0
4,cl80a0k6q0hs2075gctis6fn2,4188,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,NaN,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13517,cl8plyu7i1vh007323awr2pfi,2094,2.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,-1.0
13518,cl8pm0h9d4a56080t852lbibu,2094,2.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,-1.0
13519,cl8rg3sec31un070tb5tx7d8k,4049,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,NaN,-1.0
13520,cl8rg4t2l2u0i072fcwa1fyk8,4049,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,NaN,-1.0


# Statement Level Labels

In [11]:
temp2 = df[['objects']].explode('objects').dropna().reset_index().rename({'index':'label_index'},axis=1)
temp2 = temp2.join(pd.json_normalize(temp2['objects']))
temp3 = temp2[['classifications']].explode('classifications').dropna().reset_index().rename({'index':'statement_index'},axis=1)

In [12]:
df_statement_sentiment = temp3[temp3['classifications'].apply(lambda x: type(x)==dict)].reset_index(drop=True)
df_statement_sentiment =df_statement_sentiment.join(pd.json_normalize(df_statement_sentiment['classifications']))
df_statement_sentiment= df_statement_sentiment [['statement_index',"answer"]]
df_statement_sentiment['answer'] = df_statement_sentiment['answer'].apply(lambda x: x[0]['value'] if type(x)==list else x)
df_statement_sentiment.set_index('statement_index',inplace= True)
df_statement_sentiment.rename({'answer':'statement_sentiment'},axis=1,inplace= True)

In [13]:
df_statement_aspect = temp3[temp3['classifications'].apply(lambda x: type(x)!=dict)].reset_index(drop=True)
df_statement_aspect = df_statement_aspect.explode('classifications').reset_index(drop=True)
df_statement_aspect =df_statement_aspect.join(pd.json_normalize(df_statement_aspect['classifications']))[['statement_index',"answers"]]
df_statement_aspect = df_statement_aspect.explode('answers').reset_index(drop=True)
df_statement_aspect.dropna(subset='answers',inplace= True)
df_statement_aspect['answers'] = df_statement_aspect['answers'].apply(lambda x: x['value'])
df_statement_aspect = pd.DataFrame(df_statement_aspect.groupby('statement_index')['answers'].apply(lambda x: ", ".join(list(set(x)))))

In [14]:
df_statement = temp2[['label_index','data.location.start','data.location.end']]
df_statement = df_statement.join(df_statement_sentiment)
df_statement = df_statement.join(df_statement_aspect)
df_statement.set_index('label_index',inplace=True)
df_statement = df_labelled[['ID','External ID']].join(df_statement)

In [15]:
df_statement.sort_values(['ID','External ID','data.location.start'])
df_statement['paragraphs'] = df_statement.groupby(['ID','External ID']).cumcount()
df_statement = df_statement[df_statement['paragraphs']<=15]
df_statement['paragraphs'] = df_statement['paragraphs'].apply(lambda x: f"Paragraph{x}")

In [16]:

df_statement_sentiment = df_statement.pivot(index=['ID','External ID'],columns='paragraphs',values='statement_sentiment')
df_statement_sentiment = df_statement_sentiment.replace({"positive":2,'neutral':1,'negative':0}).fillna(-1)
df_statement_aspect = df_statement.pivot(index=['ID','External ID'],columns='paragraphs',values='answers')

In [17]:
df_statement_aspect = df_statement_aspect.reindex(sorted(df_statement_aspect.columns, key = lambda x: int(x[9:])), axis=1)
cols = ["ASPECTS-"+item for item in df_statement_aspect.columns]
df_statement_aspect.columns = cols
df_statement_aspect = df_statement_aspect.reset_index()

df_statement_sentiment = df_statement_sentiment.reindex(sorted(df_statement_sentiment.columns, key = lambda x: int(x[9:])), axis=1)
cols = ["SENTIMENT-"+item for item in df_statement_sentiment.columns]
df_statement_sentiment.columns = cols
df_statement_sentiment = df_statement_sentiment.reset_index()


# Merge

In [18]:
df = pd.merge(df_article,df_statement_aspect,on=['ID', 'External ID'],how='outer')
df = pd.merge(df,df_statement_sentiment,on=['ID', 'External ID'],how='outer')

In [19]:
df.to_csv("../datasets/persent_annotated/per_sent_labelled_data.csv", index=False)